In [1]:
import utils
inp = utils.get_input(2019, 7)[:-1]
print(inp[:100])

Found cached input for 2019 day 7
3,8,1001,8,10,8,105,1,0,0,21,30,55,80,101,118,199,280,361,442,99999,3,9,101,4,9,9,4,9,99,3,9,101,4,9


## machine

In [2]:
import operator
from functools import partial
from itertools import repeat, chain
from collections import namedtuple


class IntCode:
    Op = namedtuple('Op', ['func', 'num_params', 'write_param', 'resolve_modes'])

    def __init__(self, program, pointer=0, inputs=None, **kwargs):
        self.mem = program
        self.pointer = pointer
        self.state = "started"
        self.output = None
        self._inputs = iter(inputs or [])
        self._kwargs = kwargs
        self._ops = {
            1: IntCode.Op(partial(IntCode._math, operator.add), 3, True, True),
            2: IntCode.Op(partial(IntCode._math, operator.mul), 3, True, True),
            3: IntCode.Op(IntCode._inp, 1, True, False),
            4: IntCode.Op(IntCode._out, 1, False, True),
            5: IntCode.Op(partial(IntCode._jump, lambda x: x != 0), 2, False, True),
            6: IntCode.Op(partial(IntCode._jump, lambda x: x == 0), 2, False, True),
            7: IntCode.Op(partial(IntCode._math, operator.lt), 3, True, True),
            8: IntCode.Op(partial(IntCode._math, operator.eq), 3, True, True),
            99: IntCode.Op(IntCode._halt, 0, False, False),
        }

    def with_inputs(self, inputs):
        self._inputs = chain(self._inputs, inputs)
        return self

    def run(self):
        instruction = self.mem[self.pointer]
        opcode, modes = self._opcode(instruction), self._modes(instruction)
        op = self._ops[opcode]
        params = self.mem[self.pointer + 1:self.pointer + op.num_params + 1]

        if op.resolve_modes:
            wparam = -int(op.write_param) or len(params)
            args = zip(repeat(self.mem), params[:wparam], modes[:wparam])
            params[:wparam] = map(lambda t: IntCode._resolve(*t), args)

        if self._kwargs.get('_debug', False):
            print(f"{opcode:2d}", modes, list(params))

        orig_pointer = self.pointer
        self.mem, self.pointer, output, self.state = \
            op.func(self.mem, self.pointer, *params, inputs=self._inputs, **self._kwargs)
        self.output = output if output is not None else self.output

        if self.state in ["halted", "blocked"]:
            return self

        if self.pointer == orig_pointer:
            self.pointer = self.pointer + op.num_params + 1

        return self.run()

    @staticmethod
    def _resolve(mem, val, mode):
        return mem[val] if int(mode) == 0 else val

    @staticmethod
    def _opcode(instruction):
        return int(str(instruction)[-2:])

    @staticmethod
    def _modes(instruction):
        return f"{instruction:05d}"[:3][::-1]

    @staticmethod
    def _math(func, mem, pointer, a, b, out, **kwargs):
        _mem = list(mem)
        _mem[out] = int(func(a, b))
        return _mem, pointer, None, "running"

    @staticmethod
    def _jump(func, mem, pointer, cond, val, **kwargs):
        return mem, val if func(cond) else pointer, None, "running"

    @staticmethod
    def _inp(mem, pointer, out, inputs, **kwargs):
        try:
            _mem = list(mem)
            _mem[out] = int(next(inputs))
            return _mem, pointer, None, "running"
        except StopIteration:
            return mem, pointer, None, "blocked"

    @staticmethod
    def _out(mem, pointer, val, **kwargs):
        if kwargs.get('_debug', False):
            print(f"out: {val}")
        return mem, pointer, val, "running"

    @staticmethod
    def _halt(mem, pointer, *args, **kwargs):
        return mem, pointer, None, "halted"


## part 1

In [3]:
from itertools import permutations


def part_1(program):
    Signal = namedtuple('Signal', ['value', 'setting'])
    max_signal = Signal(0, None)
    for phase in permutations('01234'):
        prev_out = 0
        for p in phase:
            amp = IntCode(program, inputs=[p, prev_out]).run()
            prev_out = amp.output
        if prev_out > max_signal.value:
            max_signal = Signal(prev_out, ''.join(phase))
    return max_signal


program = list(map(int, inp.split(',')))
print(part_1(program).value)

43812


## part 2

In [4]:
def part_2(program):
    Signal = namedtuple('Signal', ['value', 'setting'])
    max_signal = Signal(0, None)
    for phase in permutations('56789'):
        amps = [IntCode(program, inputs=[p]) for p in phase]
        prev_out = 0
        while amps[4].state != 'halted':
            for i in range(5):
                amps[i] = amps[i].with_inputs([prev_out]).run()
                prev_out = amps[i].output
        if prev_out > max_signal.value:
            max_signal = Signal(prev_out, ''.join(phase))
    return max_signal


program = list(map(int, inp.split(',')))
print(part_2(program).value)

59597414
